In [ ]:
import os
import albumentations as A
import numpy as np
from tqdm import tqdm
from PIL import Image

all_transforms = [
  A.Compose([
    A.augmentations.geometric.rotate.SafeRotate(limit=30,value=[0,0,0], border_mode=0, p=0.10),
    A.core.composition.OneOf([A.Lambda(p=1), A.VerticalFlip(p=1), A.HorizontalFlip(p=1),A.RandomRotate90(p=1)],p=1),
    A.core.composition.OneOf([A.Lambda(p=1), A.augmentations.ToGray(p=1), A.augmentations.RGBShift(p=1),
    A.augmentations.ChannelShuffle(p=1),A.augmentations.ColorJitter(p=1),A.InvertImg(p=1)],p=1),
    A.core.composition.OneOf([A.Lambda(p=1),A.augmentations.MotionBlur(p=1), A.augmentations.AdvancedBlur(p=1), A.augmentations.GaussianBlur(p=1)],p=1),
    A.ImageCompression(p=0.20, quality_lower=40),
])
]
paths= ["./train/no_watermark/","./train/watermark/"]

for p in paths:
    for file_name in tqdm(os.listdir(p)):
        img = Image.open(p+file_name)
        if img.mode != 'RGB':
           img = img.convert('RGB')
        # img=img.resize((512,512),Image.Resampling(np.random.randint(6)))    #[0,5)
        img = np.array(img)
        
        for i in range(1):  #number of augmentations per 1 image
            new_img = all_transforms[0](image=img)["image"]
        # print(new_img.shape)
        new_img =  Image.fromarray(new_img,"RGB")

        new_file_name = f'{file_name[:file_name.index(".")]}_aug_{str(i)}_{file_name[file_name.index("."):]}'
        new_img.save(p+new_file_name,quality=90)